In [13]:
!pip3 install nltk 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [4]:
!pip3 install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 715.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 382.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [7]:
!pip3 install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 400.4 kB/s eta 0:00:00:--:--

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


<h1> Différents tests </h1>

<h4> l'idée c'est de faire un code qui après le dernier poitn regarde ce qu'il ya apres, s'il n'y a qu'un mot on le remove, s'il y'a une phrase on la complete. </h4>

In [15]:
import nltk
# Télécharger explicitement toutes les ressources NLTK nécessaires
print("Téléchargement des ressources NLTK...")
nltk.download('punkt')
nltk.download('punkt_tab', quiet=True, raise_on_error=False)

Téléchargement des ressources NLTK...


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>


False

In [22]:
# Installer les paquets nécessaires
!pip install transformers sentencepiece accelerate


  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.6 MB/s eta 0:00:00a 0:00:01
Using cached huggingface_hub-0.29.3-py3-none-any.whl (468 kB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━ 54.5/66.5 MB 41.6 kB/s eta 0:04:49m
ERROR: Except

In [23]:
# Importer et configurer dans une nouvelle cellule
from transformers import pipeline

# Fonction pour compléter avec un modèle Hugging Face
def complete_with_transformers(incomplete_text):
    try:
        # Charge un petit modèle qui fonctionne sur CPU
        if not hasattr(complete_with_transformers, "generator"):
            print("Chargement du modèle (première fois seulement)...")
            complete_with_transformers.generator = pipeline('text-generation', 
                                                          model='distilgpt2',
                                                          device=-1)  # CPU
        
        # Générer la complétion
        result = complete_with_transformers.generator(incomplete_text, 
                                                    max_length=len(incomplete_text.split()) + 15,
                                                    num_return_sequences=1)[0]['generated_text']
        
        # Retourner uniquement la partie complétée
        return result
    except Exception as e:
        print(f"Erreur lors de la complétion avec Transformers: {e}")
        return incomplete_text + "."

In [24]:
# En haut du notebook
%autosave 60

Autosaving every 60 seconds


In [25]:
import psutil
def check_memory():
    print(f"Mémoire utilisée: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB")

# Appelez périodiquement dans votre boucle
if i % 100 == 0:
    check_memory()

Mémoire utilisée: 111.08 MB


In [27]:
import json
import re
import time
from tqdm import tqdm  # Version standard (pas .notebook)
import requests
import os

# Configuration
input_file = "subset.json"
output_file = "result.json"
batch_size = 10  # Réduit pour sauvegarder plus fréquemment

# Fonction pour compter les mots
def count_words(text):
    words = re.findall(r'\b\w+\b', text)
    return len(words)

# Fonction pour compléter avec Ollama (modèle local)
def complete_with_ollama(incomplete_text):
    try:
        response = requests.post('http://localhost:11434/api/generate', 
                               json={
                                   'model': 'mistral:7b-instruct-v0.2-q4_0',
                                   'prompt': f"Complete the following sentence in a coherent and concise manner: '{incomplete_text}'",
                                   'stream': False
                               }, timeout=10)
        
        if response.status_code == 200:
            return response.json().get('response', incomplete_text + ".")
        else:
            print(f"Erreur Ollama: {response.text}")
            return incomplete_text + "."
    except Exception as e:
        print(f"Erreur lors de la complétion avec Ollama: {e}")
        return incomplete_text + "."

# Fonction pour traiter chaque résumé
def process_summary(summary):
    if not summary or "." not in summary:
        return summary
    
    try:
        main_text, last_part = summary.rsplit(".", 1)
        last_part = last_part.strip()
        
        if not last_part:
            return main_text + "."
        
        word_count = count_words(last_part)
        
        if word_count == 1:
            print(f"Suppression du mot '{last_part}' en fin de résumé")
            return main_text + "."
        
        elif word_count > 1:
            print(f"Complétion de la phrase: '{last_part}'")
            completed_sentence = complete_with_ollama(last_part)
            return main_text + ". " + completed_sentence
        
        else:
            return main_text + "."
    except Exception as e:
        print(f"Erreur lors du traitement du résumé: {e}")
        return summary

# Fonction pour sauvegarder la progression
def save_progress(current_index):
    with open("progress.txt", "w") as f:
        f.write(str(current_index))

# Fonction pour charger la progression
def load_progress():
    try:
        with open("progress.txt", "r") as f:
            return int(f.read().strip())
    except:
        return 0

# Fonction principale - pour meilleure organisation
def main():
    start_time = time.time()

    try:
        # Lire le fichier JSON
        print(f"Lecture du fichier {input_file}...")
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, list):
            items = data
        else:
            items = [data]
        
        total_items = len(items)
        print(f"Traitement de {total_items} éléments...")
        
        # Variables pour les statistiques
        modified_count = 0
        removed_words = 0
        completed_phrases = 0
        errors = []
        
        # Reprendre le traitement si interrompu précédemment
        start_index = load_progress()
        if start_index > 0:
            print(f"Reprise du traitement à partir de l'élément {start_index+1}")
        
        # Traiter chaque élément - avec version standard de tqdm
        for i, item in enumerate(tqdm(items[start_index:], desc="Traitement des résumés")):
            current_index = start_index + i
            
            try:
                if "summary" in item and item["summary"]:
                    original_summary = item["summary"]
                    new_summary = process_summary(original_summary)
                    
                    if original_summary != new_summary:
                        modified_count += 1
                        
                        if len(original_summary) > len(new_summary):
                            removed_words += 1
                        else:
                            completed_phrases += 1
                    
                    item["summary"] = new_summary
            except Exception as e:
                errors.append({"index": current_index, "error": str(e)})
                print(f"Erreur sur l'élément {current_index}: {e}")
            
            # Sauvegarder la progression
            save_progress(current_index + 1)
            
            # Sauvegarde intermédiaire
            if (i + 1) % batch_size == 0:
                print(f"\nSauvegarde intermédiaire après {current_index + 1}/{total_items} éléments...")
                with open(output_file, 'w', encoding='utf-8') as f:
                    if isinstance(data, list):
                        json.dump(items, f, ensure_ascii=False, indent=2)
                    else:
                        json.dump(items[0], f, ensure_ascii=False, indent=2)
        
        # Écriture finale
        print(f"\nÉcriture du résultat dans {output_file}...")
        with open(output_file, 'w', encoding='utf-8') as f:
            if isinstance(data, list):
                json.dump(items, f, ensure_ascii=False, indent=2)
            else:
                json.dump(items[0], f, ensure_ascii=False, indent=2)
        
        # Si des erreurs ont été rencontrées, les enregistrer
        if errors:
            with open("errors.json", "w", encoding="utf-8") as f:
                json.dump(errors, f, ensure_ascii=False, indent=2)
            print(f"Des erreurs ont été rencontrées pour {len(errors)} éléments. Détails dans errors.json")
        
        elapsed_time = time.time() - start_time
        print(f"\nTraitement terminé en {elapsed_time:.2f} secondes")
        print(f"Statistiques:")
        print(f"- Résumés modifiés: {modified_count}/{total_items}")
        print(f"- Mots uniques supprimés: {removed_words}")
        print(f"- Phrases complétées: {completed_phrases}")
        
        return 0
    except Exception as e:
        print(f"Erreur générale: {e}")
        return 1

# Exécution directe
if __name__ == "__main__":
    main()
else:
    # Si importé dans un notebook
    main()

Lecture du fichier subset.json...
Traitement de 3 éléments...


Traitement des résumés: 100%|██████████| 3/3 [00:00<00:00, 587.30it/s]

Complétion de la phrase: 'Les facteurs de risque incluent le tabagisme, l'alcoolisme, les régimes alimentaires, les infections, et les polluants industriels reconnus cancérigènes'
Erreur lors de la complétion avec Ollama: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x169602270>: Failed to establish a new connection: [Errno 61] Connection refused'))
Suppression du mot 'Réponse:' en fin de résumé
Complétion de la phrase: 'Les professionnels de la santé recommandent des rapports protég'
Erreur lors de la complétion avec Ollama: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16aaa9d10>: Failed to establish a new connection: [Errno 61] Connection refused'))

Écriture du résultat dans result.json...

Traitement terminé en 0.01 secondes
Statist

In [33]:
# #!/usr/bin/env python3
# import json
# import re
# import time
# from tqdm import tqdm
# from openai import OpenAI

# # Configuration
# input_file = "subset.json"
# output_file = "result.json"
# api_key = 
# batch_size = 50

# # Fonction simple pour compter les mots (sans utiliser NLTK)
# def count_words(text):
#     """Compte les mots dans un texte en utilisant des expressions régulières"""
#     words = re.findall(r'\b\w+\b', text)
#     return len(words)

# def complete_with_local_model(incomplete_text):
#     from transformers import pipeline
    
#     # Charge un modèle plus léger qui fonctionne sur CPU
#     generator = pipeline('text-generation', model='gpt2')
#     result = generator(incomplete_text, max_length=30, num_return_sequences=1)[0]['generated_text']
    
#     # Nettoyer la sortie pour ne garder que la complétion
#     return result

# def complete_sentence_with_openai(incomplete_text, api_key):
#     """Complète une phrase incomplète en utilisant l'API OpenAI"""
#     try:
#         client = OpenAI(api_key=api_key)
        
#         response = client.completions.create(
#             model="gpt-3.5-turbo-instruct",
#             prompt=f"Complète cette phrase de manière cohérente et concise: '{incomplete_text}'",
#             max_tokens=30,
#             temperature=0.7
#         )
        
#         completed = response.choices[0].text.strip()
#         return completed
#     except Exception as e:
#         print(f"Erreur lors de la complétion OpenAI: {e}")
#         # Fallback: retourner le texte avec un point final
#         return incomplete_text + "."

# def process_summary(summary, api_key):
#     """
#     Traite le champ summary selon les règles définies:
#     1. Si après le dernier point il n'y a qu'un seul mot, le supprimer
#     2. Si après le dernier point il y a plusieurs mots (phrase incomplète), la compléter
#     """
#     # Si le texte est vide ou ne contient pas de point
#     if not summary or "." not in summary:
#         return summary
    
#     try:
#         # Séparer le texte en fonction du dernier point
#         main_text, last_part = summary.rsplit(".", 1)
        
#         # Nettoyer la dernière partie
#         last_part = last_part.strip()
        
#         # Si la dernière partie est vide, retourner le texte principal avec un point
#         if not last_part:
#             return main_text + "."
        
#         # Compter les mots dans la dernière partie - SANS NLTK
#         word_count = count_words(last_part)
        
#         # Si un seul mot, le supprimer
#         if word_count == 1:
#             print(f"Suppression du mot '{last_part}' en fin de résumé")
#             return main_text + "."
        
#         # Si plusieurs mots, compléter la phrase avec OpenAI
#         elif word_count > 1:
#             print(f"Complétion de la phrase: '{last_part}'")
#             completed_sentence =  complete_with_local_model(last_part)
#             return main_text + ". " + completed_sentence
        
#         # Cas improbable, mais par sécurité
#         else:
#             return main_text + "."
#     except Exception as e:
#         print(f"Erreur lors du traitement du résumé: {e}")
#         return summary  # En cas d'erreur, retourner le résumé original

# # Fonction principale
# def main():
#     start_time = time.time()
    
#     try:
#         # Lire le fichier JSON
#         print(f"Lecture du fichier {input_file}...")
#         with open(input_file, 'r', encoding='utf-8') as f:
#             data = json.load(f)
        
#         # Vérifier le format (liste d'objets ou objet unique)
#         if isinstance(data, list):
#             items = data
#         else:
#             items = [data]  # Convertir en liste pour traitement uniforme
        
#         total_items = len(items)
#         print(f"Traitement de {total_items} éléments...")
        
#         # Variables pour les statistiques
#         modified_count = 0
#         removed_words = 0
#         completed_phrases = 0
        
#         # Traiter chaque élément avec une barre de progression
#         for i, item in enumerate(tqdm(items, desc="Traitement des résumés")):
#             if "summary" in item and item["summary"]:
#                 original_summary = item["summary"]
#                 new_summary = process_summary(original_summary, api_key)
                
#                 if original_summary != new_summary:
#                     modified_count += 1
                    
#                     # Déterminer si c'était une suppression ou une complétion
#                     if len(original_summary) > len(new_summary):
#                         removed_words += 1
#                     else:
#                         completed_phrases += 1
                
#                 item["summary"] = new_summary
            
#             # Sauvegarde intermédiaire pour éviter de perdre du travail
#             if (i + 1) % batch_size == 0 and i > 0:
#                 print(f"\nSauvegarde intermédiaire après {i + 1}/{total_items} éléments...")
#                 with open(output_file, 'w', encoding='utf-8') as f:
#                     if isinstance(data, list):
#                         json.dump(items, f, ensure_ascii=False, indent=2)
#                     else:
#                         json.dump(items[0], f, ensure_ascii=False, indent=2)
        
#         # Écrire le résultat final
#         print(f"\nÉcriture du résultat dans {output_file}...")
#         with open(output_file, 'w', encoding='utf-8') as f:
#             if isinstance(data, list):
#                 json.dump(items, f, ensure_ascii=False, indent=2)
#             else:
#                 json.dump(items[0], f, ensure_ascii=False, indent=2)
        
#         elapsed_time = time.time() - start_time
#         print(f"\nTraitement terminé en {elapsed_time:.2f} secondes")
#         print(f"Statistiques:")
#         print(f"- Résumés modifiés: {modified_count}/{total_items}")
#         print(f"- Mots uniques supprimés: {removed_words}")
#         print(f"- Phrases complétées: {completed_phrases}")
        
#         return 0
        
#     except Exception as e:
#         print(f"Erreur générale: {e}")
#         return 1

# # Exécution directe
# if __name__ == "__main__":
#     main()
# else:
#     # Si importé dans un notebook, exécuter directement
#     main()

pb avec pkuntab

In [34]:
# #!/usr/bin/env python3
# import json
# import nltk
# from nltk.tokenize import word_tokenize
# import time
# from tqdm import tqdm
# from openai import OpenAI

# # Télécharger les ressources NLTK si nécessaire
# try:
#     nltk.data.find('tokenizers/punkt')
# except LookupError:
#     nltk.download('punkt')

# # Configuration - définissez directement les variables plutôt que d'utiliser argparse
# input_file = "subset.json"  # Définir directement le fichier d'entrée
# output_file = "result.json"  # Définir directement le fichier de sortie
# batch_size = 50

# def complete_sentence_with_openai(incomplete_text, api_key):
#     """Complète une phrase incomplète en utilisant l'API OpenAI"""
#     try:
#         client = OpenAI(api_key=api_key)
        
#         response = client.completions.create(
#             model="gpt-3.5-turbo-instruct",
#             prompt=f"Complète cette phrase de manière cohérente et concise: '{incomplete_text}'",
#             max_tokens=30,
#             temperature=0.7
#         )
        
#         completed = response.choices[0].text.strip()
#         return completed
#     except Exception as e:
#         print(f"Erreur lors de la complétion OpenAI: {e}")
#         # Fallback: retourner le texte avec un point final
#         return incomplete_text + "."

# def process_summary(summary, api_key):
#     """
#     Traite le champ summary selon les règles définies:
#     1. Si après le dernier point il n'y a qu'un seul mot, le supprimer
#     2. Si après le dernier point il y a plusieurs mots (phrase incomplète), la compléter
#     """
#     # Si le texte est vide ou ne contient pas de point
#     if not summary or "." not in summary:
#         return summary
    
#     # Séparer le texte en fonction du dernier point
#     main_text, last_part = summary.rsplit(".", 1)
    
#     # Nettoyer la dernière partie
#     last_part = last_part.strip()
    
#     # Si la dernière partie est vide, retourner le texte principal avec un point
#     if not last_part:
#         return main_text + "."
    
#     # Compter les mots dans la dernière partie
#     words = word_tokenize(last_part)
#     word_count = len(words)
    
#     # Si un seul mot, le supprimer
#     if word_count == 1:
#         print(f"Suppression du mot '{last_part}' en fin de résumé")
#         return main_text + "."
    
#     # Si plusieurs mots, compléter la phrase avec OpenAI
#     elif word_count > 1:
#         print(f"Complétion de la phrase: '{last_part}'")
#         completed_sentence = complete_sentence_with_openai(last_part, api_key)
#         return main_text + ". " + completed_sentence
    
#     # Cas improbable, mais par sécurité
#     else:
#         return main_text + "."

# # Fonction principale - remplacée par code direct
# start_time = time.time()

# try:
#     # Lire le fichier JSON
#     print(f"Lecture du fichier {input_file}...")
#     with open(input_file, 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     # Vérifier le format (liste d'objets ou objet unique)
#     if isinstance(data, list):
#         items = data
#     else:
#         items = [data]  # Convertir en liste pour traitement uniforme
    
#     total_items = len(items)
#     print(f"Traitement de {total_items} éléments...")
    
#     # Variables pour les statistiques
#     modified_count = 0
#     removed_words = 0
#     completed_phrases = 0
    
#     # Traiter chaque élément avec une barre de progression
#     for i, item in enumerate(tqdm(items, desc="Traitement des résumés")):
#         if "summary" in item and item["summary"]:
#             original_summary = item["summary"]
#             new_summary = process_summary(original_summary, api_key)
            
#             if original_summary != new_summary:
#                 modified_count += 1
                
#                 # Déterminer si c'était une suppression ou une complétion
#                 if len(original_summary) > len(new_summary):
#                     removed_words += 1
#                 else:
#                     completed_phrases += 1
            
#             item["summary"] = new_summary
        
#         # Sauvegarde intermédiaire pour éviter de perdre du travail
#         if (i + 1) % batch_size == 0:
#             print(f"\nSauvegarde intermédiaire après {i + 1}/{total_items} éléments...")
#             with open(output_file, 'w', encoding='utf-8') as f:
#                 if isinstance(data, list):
#                     json.dump(items, f, ensure_ascii=False, indent=2)
#                 else:
#                     json.dump(items[0], f, ensure_ascii=False, indent=2)
    
#     # Écrire le résultat final
#     print(f"\nÉcriture du résultat dans {output_file}...")
#     with open(output_file, 'w', encoding='utf-8') as f:
#         if isinstance(data, list):
#             json.dump(items, f, ensure_ascii=False, indent=2)
#         else:
#             json.dump(items[0], f, ensure_ascii=False, indent=2)
    
#     elapsed_time = time.time() - start_time
#     print(f"\nTraitement terminé en {elapsed_time:.2f} secondes")
#     print(f"Statistiques:")
#     print(f"- Résumés modifiés: {modified_count}/{total_items}")
#     print(f"- Mots uniques supprimés: {removed_words}")
#     print(f"- Phrases complétées: {completed_phrases}")
    
# except Exception as e:
#     print(f"Erreur: {e}")

<h1> Juste enlever les phrases incomplètes du dataset </h1>

In [30]:
import json
from tqdm import tqdm
import time

# Configuration
input_file = "medical_summaries_tronqués.json"
output_file = "result_clean_5000.json"

def clean_summary(summary):
    """Supprime tout ce qui suit le dernier point dans un texte"""
    if not summary or "." not in summary:
        return summary
    
    # Diviser au dernier point et ne garder que la partie avant
    main_text, _ = summary.rsplit(".", 1)
    
    # Retourner le texte principal avec un point
    return main_text + "."

def main():
    start_time = time.time()
    
    try:
        # Lire le fichier JSON
        print(f"Lecture du fichier {input_file}...")
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Vérifier le format (liste ou objet unique)
        if isinstance(data, list):
            items = data
        else:
            items = [data]
        
        total_items = len(items)
        print(f"Traitement de {total_items} éléments...")
        
        # Statistiques
        modified_count = 0
        
        # Traiter chaque élément
        for item in tqdm(items, desc="Nettoyage des résumés"):
            if "summary" in item and item["summary"]:
                original_summary = item["summary"]
                new_summary = clean_summary(original_summary)
                
                if original_summary != new_summary:
                    modified_count += 1
                
                item["summary"] = new_summary
        
        # Écrire le résultat
        print(f"Écriture du résultat dans {output_file}...")
        with open(output_file, 'w', encoding='utf-8') as f:
            if isinstance(data, list):
                json.dump(items, f, ensure_ascii=False, indent=2)
            else:
                json.dump(items[0], f, ensure_ascii=False, indent=2)
        
        elapsed_time = time.time() - start_time
        print(f"\nTraitement terminé en {elapsed_time:.2f} secondes")
        print(f"Résumés modifiés: {modified_count}/{total_items}")
        
    except Exception as e:
        print(f"Erreur: {e}")

if __name__ == "__main__":
    main()

Lecture du fichier medical_summaries_tronqués.json...
Traitement de 5000 éléments...


Nettoyage des résumés: 100%|██████████| 5000/5000 [00:00<00:00, 1100174.17it/s]

Écriture du résultat dans result_clean_5000.json...

Traitement terminé en 0.37 secondes
Résumés modifiés: 4610/5000


<h1> Etape 3 </h1>

In [32]:
import json
import random
import os
from tqdm import tqdm

# Configuration
input_file = "result_clean_5000.json"
output_dir = "dataset_splits"
train_ratio = 0.7  # Corrigé à 70%
val_ratio = 0.1    # 10%
test_ratio = 0.2   # 20%

def main():
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Charger les données
    print(f"Chargement des données depuis {input_file}...")
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # S'assurer que les données sont dans une liste
    if not isinstance(data, list):
        data = [data]
    
    total_items = len(data)
    print(f"Jeu de données chargé: {total_items} documents")
    
    # Mélanger les données pour éviter tout biais lié à l'ordre
    print("Mélange aléatoire des données...")
    random.seed(42)  # Pour la reproductibilité
    random.shuffle(data)
    
    # Calculer les tailles des ensembles
    train_size = int(total_items * train_ratio)
    val_size = int(total_items * val_ratio)
    test_size = total_items - train_size - val_size
    
    # Diviser les données
    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]
    
    print(f"Division effectuée:")
    print(f"- Ensemble d'entraînement: {len(train_data)} documents ({train_ratio*100:.1f}%)")
    print(f"- Ensemble de validation: {len(val_data)} documents ({val_ratio*100:.1f}%)")
    print(f"- Ensemble de test: {len(test_data)} documents ({len(test_data)/total_items*100:.1f}%)")
    
    # Sauvegarder les ensembles
    print("Sauvegarde des ensembles...")
    
    with open(os.path.join(output_dir, "train.json"), 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=2)
    
    with open(os.path.join(output_dir, "validation.json"), 'w', encoding='utf-8') as f:
        json.dump(val_data, f, ensure_ascii=False, indent=2)
    
    with open(os.path.join(output_dir, "test.json"), 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=2)
    
    # Créer également des versions CSV pour plus de flexibilité
    try:
        import pandas as pd
        
        print("Création de versions CSV des ensembles...")
        
        # Fonction pour convertir en DataFrame
        def convert_to_df(data_list):
            # Adaptation selon la structure de vos données
            # Supposant que chaque élément a un champ 'text' et 'summary'
            rows = []
            for item in data_list:
                row = {}
                # Ajoutez ici tous les champs que vous voulez conserver
                if 'text' in item:
                    row['text'] = item['text']
                if 'summary' in item:
                    row['summary'] = item['summary']
                # Ajoutez d'autres champs au besoin
                rows.append(row)
            return pd.DataFrame(rows)
        
        # Convertir et sauvegarder
        convert_to_df(train_data).to_csv(os.path.join(output_dir, "train.csv"), index=False)
        convert_to_df(val_data).to_csv(os.path.join(output_dir, "validation.csv"), index=False)
        convert_to_df(test_data).to_csv(os.path.join(output_dir, "test.csv"), index=False)
        
    except ImportError:
        print("Module pandas non disponible. Création des fichiers CSV ignorée.")
    
    print("\nDivision des données terminée avec succès!")
    print(f"Les fichiers ont été sauvegardés dans le répertoire '{output_dir}'")
    
    # Statistiques optionnelles
    print("\nStatistiques des ensembles:")
    
    def count_tokens(text):
        # Méthode simple pour estimer le nombre de tokens
        return len(text.split())
    
    for split_name, dataset in [("Train", train_data), ("Validation", val_data), ("Test", test_data)]:
        try:
            # Calculer des statistiques sur les longueurs de texte et de résumé
            text_lengths = [count_tokens(item.get('text', '')) for item in dataset if 'text' in item]
            summary_lengths = [count_tokens(item.get('summary', '')) for item in dataset if 'summary' in item]
            
            if text_lengths:
                print(f"{split_name} - Longueur moyenne des textes: {sum(text_lengths)/len(text_lengths):.1f} tokens")
                print(f"{split_name} - Texte min/max: {min(text_lengths)}/{max(text_lengths)} tokens")
            if summary_lengths:
                print(f"{split_name} - Longueur moyenne des résumés: {sum(summary_lengths)/len(summary_lengths):.1f} tokens")
                print(f"{split_name} - Résumé min/max: {min(summary_lengths)}/{max(summary_lengths)} tokens")
        except Exception as e:
            print(f"Impossible de calculer les statistiques pour l'ensemble {split_name}: {e}")

if __name__ == "__main__":
    main()

Chargement des données depuis result_clean_5000.json...
Jeu de données chargé: 5000 documents
Mélange aléatoire des données...
Division effectuée:
- Ensemble d'entraînement: 3500 documents (70.0%)
- Ensemble de validation: 500 documents (10.0%)
- Ensemble de test: 1000 documents (20.0%)
Sauvegarde des ensembles...
Création de versions CSV des ensembles...

Division des données terminée avec succès!
Les fichiers ont été sauvegardés dans le répertoire 'dataset_splits'

Statistiques des ensembles:
Train - Longueur moyenne des résumés: 94.7 tokens
Train - Résumé min/max: 0/1073 tokens
Validation - Longueur moyenne des résumés: 96.8 tokens
Validation - Résumé min/max: 0/678 tokens
Test - Longueur moyenne des résumés: 94.7 tokens
Test - Résumé min/max: 0/497 tokens
